In [ ]:
import pandas as pd # data processing 
import os # access to dir structure
import matplotlib.pyplot as plt # plotting 
import numpy as np # linear algebra 
import seaborn as sns 

# import kaggle 

Dataset in use 
https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset?select=True.csv

In [ ]:
# view files in downloaded dataset folder 

csv_files = os.listdir(r"/Users/Taurai/Iza/CFG/Data Science : Part 2/Project Folder /archive(3)")
for file in csv_files: 
    print(file)
print(csv_files.count)

In [ ]:
fake_df1 = pd.read_csv("./archive(3)/Fake.csv")
real_df1 = pd.read_csv("./archive(3)/True.csv")

In [ ]:
fake_df1.head(10)

In [ ]:
real_df1.head(20)

In [ ]:
real_df1['subject'].value_counts

In [ ]:
fake_df1['subject'].unique()

In [ ]:
print(fake_df1.shape)
print(real_df1.shape)

In [ ]:
# label the diff df for easier ref

fake_df1['true'] = 0
real_df1['true'] = 1

# print new shape 
print(fake_df1.shape)
print(real_df1.shape)

In [ ]:
# just to see 

real_df1.describe


In [ ]:
# concat the two datasets  

raw_df = pd.concat([fake_df1, real_df1])

In [ ]:
raw_df

In [ ]:
raw_df.true.value_counts 

In [ ]:
# counting by subject

for key, count in raw_df['subject'].value_counts().iteritems():
    print(f"{key}:\t{count}")

In [ ]:
# spread of information sources/ subjects in raw df 

plt.figure(figsize=(10,6))
sns.countplot('subject', data = raw_df)
plt.show()

In [ ]:
# move until found the right place to position/ when needed import plotly.express as px

from wordcloud import WordCloud, STOPWORDS

# colored output text
from termcolor import colored
import re

# display and store Matplotlib plots within a Python Jupyter notebook (no need to use .show())
%matplotlib inline

In [ ]:
# word cloud of fake vs real df for quick comparison before cleaning and processing 

from wordcloud import WordCloud, STOPWORDS
import nltk


text = ''
for news in fake_df1.text.values:
    text += f" {news}"
wordcloud = WordCloud(
    width = 750, height = 400, 
    background_color = 'purple', 
    stopwords=set(nltk.corpus.stopwords.words("english"))).generate(text)
fig = plt.figure(
    figsize = (20,15), 
    facecolor = 'm',
    edgecolor = 'm')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.show
del text

In [ ]:
# Real news word cloud 

text = ''
for news in real_df1.text.values:
    text += f" {news}"
wordcloud = WordCloud(
    width = 750, height = 400, 
    background_color = 'white', 
    stopwords=set(nltk.corpus.stopwords.words("english"))).generate(text)
fig = plt.figure(
    figsize = (20,15), 
    facecolor = 'm',
    edgecolor = 'm')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.show
del text

## Data Cleaning 

Df needs cleaning. Ultimate Goal is:  
* Check for null values
* check for empty spaces 
* check for nonsense texts 
* remove urls, https, special characters? 
* remove REUTERS()
* group text by subject? and label


In [ ]:
# check raw_df starting point
raw_df.shape

In [ ]:
# check null values
raw_df.isnull().sum()

In [ ]:
# drop missing values

raw_df = raw_df.dropna()
raw_df.shape

# Note, dropna() not picking up null values as some text missing in DF. 

In [ ]:
# Make [index] for rows that don't have publication info (REUTERS)
# note: not all - are reuters, and may be in actual text


no_publisher = []
for index, row in enumerate(raw_df.text.values):
    try:
        record = row.split(" -", maxsplit=1)
        
        # if no text present, this should raise error 
        record[1]
        assert(len(record[0]) < 260)
    except:
        no_publisher.append(index)

In [ ]:
# check for the 'clean' data, that doesn't start with REUTERS
# note 23445 records

raw_df.iloc[no_publisher].text

In [ ]:
# create list of indexes for rows that have publication info (REUTERS)
# this will seperate the text from the publisher info (i.e seperate REUTERS())

publisher = []
pt_text=[]

for index, row in enumerate(raw_df.text.values):
    if index in no_publisher: 
        
#         if no publisher mentioned, add unknown
        pt_text.append(row)
        publisher.append("Unknown")
        continue 
        
    record = row.split(" -", maxsplit=1)
    publisher.append(record[0])
    pt_text.append(record[1])

In [ ]:
# replace current text column with the new one made above 
# add new seperate column for publisher info (i.e REUTERS())

raw_df["publisher"] = publisher 
raw_df["text"] = pt_text

del publisher, pt_text, record, no_publisher

In [ ]:
raw_df.head(10)

In [ ]:
raw_df.tail(15)

In [ ]:
# check for empty rows and list them
#  note 631 empty values 

empty = [index for index, text in enumerate(raw_df.text.values) if str(text).strip()=='']
print(f"Number of empty rows: {len(empty)}")
raw_df.iloc[empty].tail()

In [ ]:
#  drop these empty row records 

# raw_df = raw_df.drop(empty, axis=0)
# raw_df = raw_df[raw_df['text'].notna()]

In [ ]:
# check empty values dropped
# note still present in df 
raw_df.shape

### Why isn't dropna working?  

it now is after adding white space to empty text str.

In [ ]:
# use numpy to replace white space with NaN function in order to pick up using dropna()

raw_df['text'].replace(' ', np.nan, inplace=True)

In [ ]:
# confirm no empty rows 
# use drop method to check for empty spaces and not white spaces 
# note lost some values 

raw_df = raw_df.drop([index for index, text in enumerate(raw_df.text.values) if str(text).strip()==''])
raw_df.shape

In [ ]:
# now can drop the null values using dropna 
# note lost more values  

raw_df = raw_df.dropna(subset=['text'])
raw_df.shape

In [ ]:
raw_df

In [ ]:
# text has bool true so should return false if empty 
# note DF length matches above with 44263 rows

raw_df['text'].astype(bool)

In [ ]:
# raw_df = raw_df.drop(empty, inplace=True)
# raw_df['text'].str.strip().astype(bool)
# raw_df['text'].astype(bool)
# df.dropna()
# raw_df.iloc[empty].tail()
# raw_df = (raw_df.iloc[empty]).replace('', np.nan)
# print(raw_df)
# print(type(df))
# raw_df.empty

## group by subject and label and analyze those 
## create histograms for subjects 
## create text count for label? or show article distributions?

## Let's clean up the DF 

Delete unecessary rows - publisher, date, title, subject 
They won't be needed going forwards


In [ ]:
del raw_df['title']
del raw_df['subject']
del raw_df['date']
del raw_df['publisher']


raw_df.head(10)

### Lets Clean up the data

* Remove punctuation
* remove special char 
* convert upper to lower 
* remove stopwords
* remove urls
* Lemmatization - The stemming of words without loss of meaning to context


We'll clean the first news article only for now... 

In [ ]:
text_1 = raw_df['text'][0]
type(text_1)

In [ ]:
#  use contractions lib for context when expanding contractions (i'd -> i would)
# import contractions 

In [ ]:
text_1 = text_1.to_string()
type(text_1)

In [ ]:
# raw_df['text'] = raw_df['text'].astype(str)

In [ ]:
# contractions now working after comnverting to str 
text_1 = ' '.join([contractions.fix(word) for word in text_1.split()])
text_1



In [ ]:
# will tokenizer work instead? as a secondary solution
# apply word_tokenize function 

"""


tokens = raw_df['text'].apply(word_tokenize)
tokens 

or 


"""

In [ ]:
#  remove stopwords 
#  is stopword library inclusive enough? 


from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

text_1 = nltk.word_tokenize(text_1) 
text_1 = [ word for word in text_1 if not word in set(stopwords.words("english"))]


In [ ]:
text_1 = ' '.join(text_1)
text_1

In [ ]:
# Remove special characters and punctuation 
import re 
import string 

text_1 = re.sub('\[[^]]*\]', ' ', text_1)
text_1 = re.sub('[^a-zA-Z]', ' ', text_1)

#  convert from  upper to lower 
text_1 = text_1.lower()

text_1

In [ ]:
#  removal of HTML content

from bs4 import BeautifulSoup

soup = BeautifulSoup(text_1, "html.parser")
text_1 = soup.get_text()
text_1

In [ ]:
# Lemmatization 

lemma = nltk.WordNetLemmatizer()
text_1 = [lemma.lemmatize(word) for word in text_1.split()]

text_1 = " ".join(text_1)
text_1

* perhaps remove numbers? 
* what about exclamation and question marks? 

### Now lets use the tested cleansing above and make functions to  cleanse all text. add a few more functions

In [ ]:
# remove URL
def remove_URL(text):
    url = re.compile(r'https?://\S+')
    return url.sub(r' httpsmark ', text)

# HTML removal 
def remove_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()


# Remove punctuation
def remove_punct(text):
    return re.sub('\[[^]]*\]', ' ', text)


# remove special chars
def remove_chars(text):
    return re.sub("[^a-zA-A]", " ", text)

# remove emoji 
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r' emoji ', string)


# remove stopwords
def remove_stop_and_lemmat(text): 
    final_text = []
    text = text.lower()
    text = nltk.word_tokenize(text)
    
    for word in text:
        if word not in set(stopwords.words('english')):
            lemma = nltk.WordNetLemmatizer()
            word = lemma.lemmatize(word)
            final_text.append(word)
    return " ".join(final_text)


# full function 
def cleanse(text):
    text = remove_URL(text)
    text = remove_html(text)
    text = remove_punct(text)
    text = remove_chars(text)
    text = remove_emoji(text)
    text = remove_stop_and_lemmat(text)
    


In [ ]:
# Apply function to text col
raw_df['text']=raw_df['text'].apply(cleanse)

In [ ]:
raw_df.head()

In [ ]:
# word cloud now with new cleaned data 
# STOPWORDS =set(nltk.corpus.stopwords.words("english")).generate(text)

plt.figure(figsize = (10,6))
wc = WordCloud(width = 440, 
               height = 200, 
               stopwords =set(nltk.corpus.stopwords.words("english")).generate(" ".join(raw_df['text']))
plt.imshow(wc, interpolation = 'bilinear')

In [ ]:
raw_df.tail(15)


# sentiment analysis - says if dataset is positive, neg or neutral
# top to beck groups text into categories (topic model?)1
# dashboard for visualisation (powerbi)
# pull out a few words from cloud when describing analysis
# use time frames to compare e.g. most pop topic in feb 2020 was
# 30% train and 70% test for ML 
# topic model